In [1]:
import numpy as np
import os

In [2]:
COREF_LOGS = '/mnt/data0/dtadimeti/manosphere/logs/'
COREF_REDDIT = COREF_LOGS + 'coref_reddit/'

In [14]:
fem = set(['she', 'her', 'hers', 'herself'])
masc = set(['he', 'him', 'his', 'himself'])
ratios = {} # {year-month : fem frac}
all_fem_count = 0 # total
all_masc_count = 0 # total
boyfriend_lines = [] # clusters that contain 'boyfriend' substring
for year_month in os.listdir(COREF_REDDIT): 
    with open(COREF_REDDIT + year_month, 'r') as infile: 
        for line in infile: 
            # very crude check
            if 'boyfriend' in line: 
                contents = line.split('\t')
                for clust in contents: 
                    if 'boyfriend' in clust: 
                        boyfriend_lines.append(clust)
    fem_count = 0
    masc_count = 0
    
    boyfriend_count = 0
    for clust in boyfriend_lines: 
        items = set(clust.split('$'))
        # is a cluster that contains 'boyfriend' as a span or proceeded by one other token
        is_boyfriend_cluster = False
        for item in items: 
            if item.split(' ')[-1] == 'boyfriend' and len(item.split(' ')) <= 2: 
                is_boyfriend_cluster = True
                boyfriend_count += 1
        if is_boyfriend_cluster: 
            # only count it once per cluster
            if items & fem: 
                fem_count += 1
            # only count it once per cluster
            if items & masc: 
                masc_count += 1
    if (fem_count + masc_count) != 0: 
        ratios[year_month] = fem_count / (fem_count + masc_count)
    all_fem_count += fem_count
    all_masc_count += masc_count
# print(ratios)
print(boyfriend_count)

43664


In [12]:
sorted(list(ratios.values()))

[0.4,
 0.45901639344262296,
 0.46934865900383144,
 0.47337278106508873,
 0.47337278106508873,
 0.48324514991181655,
 0.48498845265588914,
 0.48653198653198654,
 0.491679273827534,
 0.491679273827534,
 0.49522292993630573,
 0.4953560371517028,
 0.4958904109589041,
 0.49786628733997157,
 0.4986449864498645,
 0.4986449864498645,
 0.4986449864498645,
 0.5,
 0.5,
 0.5006802721088436,
 0.5025974025974026,
 0.5025974025974026,
 0.5025974025974026,
 0.5029585798816568,
 0.5051546391752577,
 0.506900878293601,
 0.506900878293601,
 0.5092592592592593,
 0.5108958837772397,
 0.5152671755725191,
 0.5154994259471871,
 0.5154994259471871,
 0.515625,
 0.5156431054461182,
 0.5213963963963963,
 0.5213963963963963,
 0.5219842164599775,
 0.5220338983050847,
 0.5229050279329609,
 0.5229050279329609,
 0.5230078563411896,
 0.5234899328859061,
 0.5234899328859061,
 0.5234899328859061,
 0.5280678851174935,
 0.5280678851174935,
 0.5286831812255541,
 0.5286831812255541,
 0.5286831812255541,
 0.5287581699346405,


In [10]:
all_fem_count / (all_fem_count + all_masc_count)

0.7902843063755952